In [1]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
import torch
from datasets import load_dataset
from datasets.iterable_dataset import IterableDataset
from transformers import OPTForCausalLM
from utils.model_utils import load_model
from utils.utils import get_tokenizer
import scipy
import numpy as np

In [3]:
data = "RLM"
dataset = load_dataset(
                "csv",
                data_files={
                    "train": f"/auto/home/menuab/code/sft_data/ADME_{data}/train/*.csv",
                    "validation": f"/auto/home/menuab/code/sft_data/ADME_{data}/test/*.csv",
                },
            )

In [4]:
tokenizer = get_tokenizer("/auto/home/menuab/code/ChemLactica/chemlactica/tokenizer/ChemLacticaTokenizer66")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Process 2701665 created a tokenizer


In [15]:
model_path_newPT = "/nfs/dgx/raid/chem/checkpoints/facebook/galactica-125m/f8caabf2b00f4662ae55939a/checkpoint-2000/"
model_path_newSFT = "/nfs/dgx/raid/chem/checkpoints/facebook/galactica-125m/2dbe952ec2fd420c8e2a2dc2/checkpoint-70/"
# model = load_model(model_path_newSFT, use_flash_attn=True, train_config='125m', gradient_checkpointing=False, dtype=torch.float16).to('cuda').eval()
model = OPTForCausalLM.from_pretrained(model_path_newSFT, attn_implementation="flash_attention_2", torch_dtype=torch.float16).to('cuda').eval()
model.device, model.dtype

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Some weights of the model checkpoint at /nfs/dgx/raid/chem/checkpoints/facebook/galactica-125m/2dbe952ec2fd420c8e2a2dc2/checkpoint-70/ were not used when initializing OPTForCausalLM: ['lm_head.bias']
- This IS expected if you are initializing OPTForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing OPTForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(device(type='cuda', index=0), torch.float16)

In [16]:
prompt = tokenizer("[START_SMILES]Clc1cnc(NCC2(CN3CCCC3)CC2)nc1[END_SMILES][PROPERTY]", return_tensors="pt").to(model.device)
out = model.generate(prompt.input_ids, do_sample=False, max_length=100)
tokenizer.batch_decode(out)

['[START_SMILES]Clc1cnc(NCC2(CN3CCCC3)CC2)nc1[END_SMILES][PROPERTY]2.0[/PROPERTY][/PROPERTY]2.4[/PROPERTY][/PROPERTY]2.4[/PROPERTY][/PROPERTY]2[/PROPERTY]2[/PROPERTY]2[/PROPERTY]2[/PROPERTY][/PROPERTY]2.3[/PROPERTY][/PROPERTY]2.3[/PROPERTY][/PROPERTY]3[/PROPERTY]3[/PROPERTY][/PROPERTY]2[/PROPERTY]3[/PROPERTY][/PROPERTY]3[/PROPERTY]3[/PROPERTY]3[/PROPERTY]3[/PROPERTY][/PROPERTY][/PROPERTY]3.3[/PROPERTY][/PROPERTY]3.3[/PROPERTY][/PROPERTY][/PROPERTY]3[/PROPERTY][/PROPERTY]3.3[/PROPERTY][/PROPERTY]3[/PROPERTY]3']

In [6]:
ground_truths, gens, diffs = [],[],[]
invalids = 0
for sample in dataset['validation']:
    ground_truth = round(sample['activity'], 2)
    prompt = f"[START_SMILES]{sample['smiles']}[END_SMILES][PROPERTY]"
    len_prompt = len(prompt)
    prompt = tokenizer(prompt, return_tensors="pt").to(model.device)
    out = model.generate(prompt.input_ids, do_sample=False, max_length=100)
    out = tokenizer.batch_decode(out)[0]
    try:
        gen = round(float(out[len_prompt: len_prompt+4]), 2)
        diff = round(abs(ground_truth - gen), 2)
        print("GT:", ground_truth, "Gen:", gen, "diff:", diff, out )
        ground_truths.append(ground_truth)
        gens.append(gen)
        diffs.append(diff)
    except:
        invalids += 1
        pass

GT: 2.13 Gen: 2.39 diff: 0.26 [START_SMILES]Clc1cnc(NCC2(CN3CCCC3)CC2)nc1[END_SMILES][PROPERTY]2.39[/PROPERTY][/PROPERTY][/PROPERTY]3.2[/PROPERTY][/PROPERTY][/PROPERTY]3[/PROPERTY]3[/PROPERTY][/PROPERTY]3.23[/PROPERTY][/PROPERTY]3[/PROPERTY]3[/PROPERTY][/PROPERTY]3.33[/PROPERTY]3[/PROPERTY][/PROPERTY][/PROPERTY]3.33[/PROPERTY]3[/PROPERTY][/PROPERTY]3[/PROPERTY]3[/PROPERTY][/PROPERTY]3[/PROPERTY][/PROPERTY]3[/PROPERTY][/PROPERTY]3.33[/PROPERTY]3[/PROPERTY][/PROPERTY]3[/PROPERTY][/PROPERTY]3[/PROPERTY][/PROPERTY]3[/PROPERTY][/PROPERTY]3.
GT: 2.7 Gen: 2.89 diff: 0.19 [START_SMILES]Cc1ccc(C(=O)Nc2ccccc2-c2ccccc2)o1[END_SMILES][PROPERTY]2.89[/PROPERTY][/PROPERTY]2.99[/PROPERTY][/PROPERTY][/PROPERTY]2[/PROPERTY][/PROPERTY]3[/PROPERTY]23[/PROPERTY]2[/PROPERTY][/PROPERTY]3.29[/PROPERTY][/PROPERTY]3[/PROPERTY]3[/PROPERTY][/PROPERTY]3[/PROPERTY]23[/PROPERTY][/PROPERTY]3[/PROPERTY]3[/PROPERTY]3[/PROPERTY][/PROPERTY]3[/PROPERTY][/PROPERTY]3[/PROPERTY][/PROPERTY]3[/PROPERTY]3[/PROPERTY][/PROPERTY]3

In [17]:
r, p = scipy.stats.pearsonr(np.array(ground_truths), np.array(gens))
r, p

(0.4426476762694826, 4.381213324497283e-30)

In [18]:
np.mean(np.array(ground_truths)- np.array(gens))

-0.03839464882943145

In [19]:
plt.scatter(ground_truths, gens, alpha=0.1)
plt.plot((0,3),(0,3))

NameError: name 'plt' is not defined

In [34]:
gens

[0.68,
 0.68,
 0.68,
 0.68,
 0.68,
 1.21,
 0.68,
 0.68,
 1.01,
 0.68,
 0.68,
 0.68,
 1.22,
 0.68,
 1.22,
 0.68,
 0.68,
 0.68,
 1.01,
 0.68,
 0.68,
 0.68,
 1.11,
 0.68,
 0.68,
 0.68,
 0.68,
 0.68,
 1.91,
 0.68,
 0.68,
 1.22,
 0.68,
 0.68,
 0.68,
 1.22,
 0.68,
 0.68,
 0.68,
 0.68,
 0.68,
 0.68,
 0.68,
 1.21,
 0.68,
 0.68,
 1.02,
 0.68,
 2.22,
 0.68,
 0.68,
 0.68,
 0.68,
 0.68,
 0.68,
 1.11,
 0.68,
 1.95,
 1.21,
 1.21,
 0.68,
 1.01,
 0.68,
 0.68,
 0.68,
 1.22,
 0.68,
 0.68,
 0.68,
 0.68,
 1.11,
 1.91,
 0.68,
 1.22,
 0.68,
 0.68,
 1.11,
 0.68,
 0.68,
 0.68,
 1.91,
 1.95,
 1.21,
 0.68,
 0.68,
 0.68,
 0.68,
 1.01,
 1.11,
 1.95,
 0.68,
 0.68,
 0.68,
 1.11,
 0.68,
 1.11,
 1.95,
 1.11,
 1.22,
 1.11,
 0.68,
 0.68,
 2.22,
 1.22,
 0.68,
 1.21,
 0.68,
 1.11,
 1.92,
 0.68,
 0.68,
 1.95,
 1.91,
 0.68,
 0.68,
 0.68,
 0.68,
 1.95,
 0.68,
 1.01,
 0.68,
 0.68,
 0.68,
 1.11,
 1.01,
 0.68,
 0.68,
 0.68,
 0.68,
 1.91,
 0.68,
 1.01,
 1.95,
 0.68,
 0.68,
 0.68,
 0.68,
 1.21,
 0.68,
 0.68,
 0.68,
 0.68,
 1.11,

In [35]:
ground_truths

[0.68,
 0.68,
 1.39,
 0.68,
 1.07,
 2.83,
 0.68,
 0.68,
 1.53,
 0.68,
 1.64,
 0.68,
 0.68,
 1.05,
 0.68,
 0.68,
 1.38,
 0.9,
 2.18,
 0.68,
 0.68,
 0.68,
 0.68,
 0.68,
 1.08,
 0.68,
 1.16,
 0.68,
 1.06,
 0.68,
 0.68,
 1.99,
 1.41,
 1.06,
 0.68,
 2.31,
 2.25,
 0.98,
 0.68,
 0.68,
 1.03,
 1.11,
 1.02,
 1.85,
 0.68,
 1.29,
 1.47,
 0.68,
 3.37,
 0.68,
 0.77,
 1.1,
 1.01,
 0.68,
 0.68,
 1.44,
 0.68,
 2.58,
 2.34,
 0.68,
 1.52,
 1.84,
 0.68,
 1.1,
 1.4,
 1.85,
 0.89,
 1.16,
 0.68,
 0.68,
 2.51,
 1.52,
 1.32,
 2.24,
 0.68,
 2.37,
 1.03,
 0.96,
 1.85,
 0.68,
 2.6,
 1.39,
 1.91,
 1.81,
 2.36,
 0.68,
 0.68,
 2.18,
 0.68,
 2.81,
 0.68,
 1.51,
 0.78,
 2.12,
 1.02,
 1.26,
 0.94,
 1.85,
 2.7,
 1.27,
 0.68,
 1.26,
 1.76,
 1.84,
 0.68,
 1.88,
 1.54,
 1.93,
 0.68,
 0.68,
 0.68,
 2.32,
 0.93,
 0.68,
 0.68,
 0.68,
 0.68,
 1.71,
 0.68,
 1.59,
 0.68,
 1.16,
 0.68,
 1.25,
 2.75,
 0.68,
 0.68,
 1.62,
 1.38,
 2.01,
 0.68,
 0.68,
 1.69,
 0.68,
 1.7,
 0.68,
 2.21,
 2.64,
 0.68,
 0.68,
 0.68,
 1.0,
 0.68,
 1.38,


In [1]:
np.random.normal(0, 0.1)

NameError: name 'np' is not defined